# 07 Build Master Table

# **Project:** NORI  
**Author:** Yuseof J  
**Date:** 10/12/25

### **Purpose**
Join socioeconomic features (extracted from tract-level government sources) on tract ID

### **Inputs**
- `data/raw/tiger_tracts_ny/tl_2025_36_tract.shp`

### **Outputs**
- `data/processed/nyc_tracts.gpkg`
  
--------------------------------------------------------------------------

### 0. Imports and Setup

In [3]:
# package imports
import os
import pandas as pd
import geopandas as gpd
from pathlib import Path

# specify filepaths
path_nyc_tracts = 'data/processed/nyc_tracts.gpkg'
path_cdc_nyc = 'data/processed/cdc_places_nyc.parquet'
#path_opp_atlas = ???
#path_acs_nyc = ???
path_output_master_table = 'data/processed/master_tract_table.parquet'

# ensure cwd is project root for file paths to function properly
project_root = Path(os.getcwd())            # get current directory
while not (project_root / "data").exists(): # keep moving up until in parent
    project_root = project_root.parent
os.chdir(project_root)                      # switch to parent directory

### 1. Load Data

In [4]:
# nyc tracts
gdf_nyc_tracts = gpd.read_file(path_nyc_tracts)

# cdc places nyc
df_cdc_nyc = pd.read_parquet(path_cdc_nyc)

In [6]:
df_cdc_nyc.head()

,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,TotalPop18plus,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,...,FOODINSECU_Crude95CI,HOUSINSECU_CrudePrev,HOUSINSECU_Crude95CI,SHUTUTILITY_CrudePrev,SHUTUTILITY_Crude95CI,LACKTRPT_CrudePrev,LACKTRPT_Crude95CI,EMOTIONSPT_CrudePrev,EMOTIONSPT_Crude95CI,Geolocation
49311,NY,New York,Bronx,36005,36005000100,3772,"3,760",23.8,"(19.1, 29.0)",12.9,...,"(51.0, 63.8)",47.3,"(42.2, 52.3)",29.6,"(26.0, 33.3)",28.6,"(24.5, 32.7)",46.9,"(42.5, 51.5)",POINT (-73.8812065 40.7929362)
49312,NY,New York,Bronx,36005,36005000200,4779,"3,714",12.2,"( 9.3, 15.6)",18.1,...,"(20.1, 29.2)",22.7,"(19.2, 26.4)",10.4,"( 8.8, 12.1)",10.7,"( 9.0, 12.6)",32.2,"(28.5, 36.3)",POINT (-73.8566781 40.8081797)
49313,NY,New York,Bronx,36005,36005000400,6272,"4,935",11.3,"( 8.4, 14.6)",17.4,...,"(17.4, 25.9)",20.6,"(17.2, 24.4)",8.9,"( 7.5, 10.5)",9.4,"( 7.8, 11.2)",32.4,"(28.4, 36.4)",POINT (-73.8514252 40.8087867)
49314,NY,New York,Bronx,36005,36005001600,5795,"4,544",15.6,"(12.2, 19.1)",23.4,...,"(31.2, 41.1)",29.8,"(26.0, 33.6)",15.2,"(13.1, 17.3)",15.4,"(13.2, 17.6)",35.1,"(31.2, 39.0)",POINT (-73.8580764 40.8188478)
49315,NY,New York,Bronx,36005,36005001901,2292,"1,725",14.3,"(11.1, 17.9)",15.1,...,"(31.3, 43.0)",33.3,"(28.7, 38.5)",18.9,"(16.0, 22.1)",17.2,"(14.6, 20.1)",35.2,"(31.3, 39.5)",POINT (-73.9280462 40.8067879)


### 2. Merge Data

In [8]:
# get nyc tract IDs
df_master = gdf_nyc_tracts[['GEOID']].copy()

# ensure GEOID is the same dtype before joins
df_master.GEOID = df_master.GEOID.astype(int)
df_cdc_nyc.TractFIPS = df_cdc_nyc.TractFIPS.astype(int)

# merge tract IDs with cdc places data
df_master = df_master.merge(df_cdc_nyc,
                            how='left',
                            left_on='GEOID',
                            right_on='TractFIPS')        

add some assertion logic - should any of this be added in previous notebooks?

assert(df_cdc_nyc tract fips are all present in geoid columns)
assert(no tract fips are missing from join)
assert(1:1 mapping between geoid and tract fips)

In [9]:
df_master.head()

,GEOID,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,TotalPop18plus,ACCESS2_CrudePrev,ACCESS2_Crude95CI,...,FOODINSECU_Crude95CI,HOUSINSECU_CrudePrev,HOUSINSECU_Crude95CI,SHUTUTILITY_CrudePrev,SHUTUTILITY_Crude95CI,LACKTRPT_CrudePrev,LACKTRPT_Crude95CI,EMOTIONSPT_CrudePrev,EMOTIONSPT_Crude95CI,Geolocation
0,36085024402,NY,New York,Richmond,36085.0,3.608502e+10,4381.0,"3,520",4.8,"( 3.6, 6.2)",...,"( 6.8, 11.8)",9.0,"( 7.0, 11.1)",4.9,"( 3.9, 6.0)",5.4,"( 4.4, 6.6)",23.9,"(20.1, 27.6)",POINT (-74.2384712 40.4997874)
1,36085027705,NY,New York,Richmond,36085.0,3.608503e+10,5866.0,"4,607",7.6,"( 6.1, 9.3)",...,"(13.2, 19.7)",13.0,"(10.8, 15.4)",6.9,"( 5.8, 8.1)",8.0,"( 6.7, 9.4)",29.2,"(25.6, 33.1)",POINT (-74.156982 40.5882479)
2,36085012806,NY,New York,Richmond,36085.0,3.608501e+10,6659.0,"5,274",8.6,"( 6.9, 10.4)",...,"(17.1, 23.9)",16.0,"(13.5, 18.5)",9.4,"( 8.0, 10.9)",10.0,"( 8.5, 11.5)",28.4,"(24.8, 32.2)",POINT (-74.1076715 40.557671)
3,36047024400,NY,New York,Kings,36047.0,3.604702e+10,3651.0,"2,184",12.5,"( 9.9, 15.3)",...,"(25.1, 35.0)",20.6,"(17.3, 23.9)",12.1,"(10.2, 14.1)",14.0,"(11.6, 16.3)",31.2,"(27.5, 34.9)",POINT (-73.9862364 40.6217475)
4,36047023000,NY,New York,Kings,36047.0,3.604702e+10,4442.0,"2,512",16.0,"(13.7, 18.4)",...,"(36.5, 44.9)",27.6,"(24.9, 30.9)",18.4,"(16.4, 20.7)",19.4,"(17.3, 21.8)",31.2,"(28.2, 34.5)",POINT (-73.9842809 40.637816)


In [ ]:
# merge master with opportunity atlas

In [ ]:
# merge master with acs data

### 3. Save Data

In [44]:
# export processed tract data
df_master.to_parquet("data/processed/")